In [2]:
from depechesAFP import getText
import gensim
import logging
import os
import json
import re
import nltk
import string
import cython
translator = str.maketrans({key: " " for key in string.punctuation})
tok = nltk.tokenize.WordPunctTokenizer()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


In [3]:
%%time
class Tweets(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        total = len(os.listdir(self.dirname))*1001
        counter = 0
        for fname in os.listdir(self.dirname):
            if counter%100 == 0:
                print(counter/total)
            for line in open(os.path.join(self.dirname, fname)).readlines():
                counter += 1
                try:
                    line = json.loads(line)
                    if "text" in line[2]:
                        #remove urls, @mentions, punctuation and uppercases
                        line = re.sub(r'https?\S+|@\S+', '',line[2]["text"]).translate(translator).lower()
                        #tokenize
                        yield tok.tokenize(line)
                except json.decoder.JSONDecodeError as error:
                    pass
                except KeyError as error:
                    pass

tweets = Tweets('/home/bmazoyer/Documents/TwitterSea/Filter')
# train the phrase detection tool from gensim to detect common bigrams like "new york"
# bigram_transformer = gensim.models.Phrases(tweets)
# train the word2vec model
# model = gensim.models.Word2Vec(bigram_transformer[tweets], workers=4)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 40.5 µs


In [22]:
bigram_transformer.save('/home/bmazoyer/Documents/TwitterSea/Word2Vec/mybigramtransformer')

In [4]:
bigram_transformer = gensim.models.Phrases.load('/home/bmazoyer/Documents/TwitterSea/Word2Vec/mybigramtransformer')

In [5]:
%%time
model = gensim.models.Word2Vec(bigram_transformer[tweets], workers=4)

0.0
0.004806536890170632
0.009613073780341264
0.014683706104037755
0.019490242994208387
0.02429677988437902
0.02910331677454965
0.03390985366472028
0.038716390554890914
0.043522927445061546
0.04832946433523218
0.05313600122540281
0.05794253811557344
0.06165427939040052
0.06646081628057114
0.07330809061162341
0.07811462750179404
0.08292116439196467
0.0877277012821353
0.09312004986121784
0.09792658675138848
0.10575341505406394
0.11055995194423457
0.1153664888344052
0.12017302572457583
0.12497956261474646
0.1321485532011848
0.13695509009135542
0.14392240780128307
0.1487289446914537
0.15353548158162433
0.15720400724005626
0.1620105441302269
0.16681708102039752
0.17162361791056815
0.1764301548007388
0.18123669169090942
0.18604322858108005
0.19084976547125068
0.1956563023614213
0.20046283925159195
0.20526937614176258
0.20706522508973843
0.2138212664528454
0.21698560992000068
0.22179214681017131
0.22659868370034195
0.23140522059051258
0.2362117574806832
0.24101829437085384
0.24341916194836166

In [39]:
model.most_similar(positive=['dylan', 'allemagne'], negative=['merkel'], topn=6)

[('françois_hollande', 0.5756078362464905),
 ('juppé', 0.5535491704940796),
 ('il', 0.5459059476852417),
 ('italie', 0.5309131741523743),
 ('sarkozy', 0.5251258611679077),
 ('islam', 0.5247024297714233)]

In [40]:
model.save('/home/bmazoyer/Documents/TwitterSea/Word2Vec/mymodel')

In [3]:
model = gensim.models.Word2Vec.load('/home/bmazoyer/Documents/TwitterSea/Word2Vec/mymodel')